# Use selected training set to create input for fitting code

In [2]:
# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic Tools
import numpy as np
import pickle
from astropy.table import Table
from astropy.io import fits
import corner
import matplotlib.pyplot as plt

# The Cannon
# import thecannon as tc

In [3]:
galah_elements = [
        'Li','C','O',
        'Na','Mg','Al','Si',
        'K','Ca','Sc','Ti','V','Cr','Mn','Co','Ni','Cu','Zn',
        'Rb','Sr','Y','Zr','Mo','Ru',
        'Ba','La','Ce','Nd','Sm','Eu'
    ]

In [4]:
training_set = Table.read('training_sets/solar_twin_training_set_16xfe_snr50.fits')
labels = np.loadtxt('training_sets/solar_twin_training_set_16xfe_snr50_labels',dtype=str)

In [5]:
# The download from datacentral also gives a list of sobject_ids for which no spectrum was available
# I have renamed them from missing_data_products to be able to identify them
sobject_id_without_spectra = np.loadtxt('dr3_spectra/missing_data_products_16xfe_snr50.txt',dtype=str,usecols=(4))
sobject_id_without_spectra = [int(sobject_id) for sobject_id in sobject_id_without_spectra]

# Let's figure out which ones do not have a spectrum on datacentral
exclude = np.ones(len(training_set),dtype=bool)
for index, sobject_id in enumerate(training_set['sobject_id']):
    if sobject_id in sobject_id_without_spectra:
        exclude[index] = False
        
# There were also spectra with issues while looping through the process
problematic_sobject_id = [150827004001397]
for sobject_id in problematic_sobject_id:
    indices = np.where(sobject_id == training_set['sobject_id'])[0]
    if len(indices) > 0:
        exclude[indices] = False

training_set = training_set[exclude]
training_set.write('training_sets/solar_twin_training_set_16xfe_snr50_with_spectra.fits',overwrite=True)

In [6]:
# Define a common wavelength grid that we will use for all spectra
# This one was used for GALAH DR2
wavelengths_for_each_ccd = dict()
wavelengths_for_each_ccd['CCD1'] = np.arange(4715.94,4896.00,0.046) # ab lines 4716.3 - 4892.3
wavelengths_for_each_ccd['CCD2'] = np.arange(5650.06,5868.25,0.055) # ab lines 5646.0 - 5867.8
wavelengths_for_each_ccd['CCD3'] = np.arange(6480.52,6733.92,0.064) # ab lines 6481.6 - 6733.4
wavelengths_for_each_ccd['CCD4'] = np.arange(7693.50,7875.55,0.074) # ab lines 7691.2 - 7838.5

wavelength_array = np.concatenate(([wavelengths_for_each_ccd['CCD'+ccd] for ccd in ['1','2','3','4']]))

wavelength_file = open('training_sets/solar_twin_training_set_16xfe_snr50_wavelength.pickle','wb')
pickle.dump((wavelength_array),wavelength_file)
wavelength_file.close()

In [104]:
# Let's create a matrix that we will later fill with the normalised flux values
normalized_flux = np.ones((np.shape(training_set)[0],np.shape(wavelength)[0]))
normalized_ivar = np.ones((np.shape(training_set)[0],np.shape(wavelength)[0]))

In [105]:
def load_normalised_spectra(sobject_id, wavelengths_for_each_ccd, spectrum_path = '/Users/svenbuder/galah_solar_twins/dr3_spectra/hermes'):
    
    # For each stars, there are 4 spectra for the 4 different CCDs.
    # We will interpolate the fluxes and uncertainties/inverse variances onto a common grid
    normalised_flux_for_index = []
    normalised_ivar_for_index = []
    # For that we first interpolate over the individual CCDs
    for ccd in ['1','2','3','4']:
        ccd_fits = fits.open('dr3_spectra/hermes/'+str(sobject_id)+ccd+'.fits')
        # Wavelengths are saved in the headers with CRVAL = starting wavelength and CDELT = delta lambda / pixels
        dr3_wavelength = ccd_fits[4].header['CRVAL1'] + ccd_fits[4].header['CDELT1']*np.arange(ccd_fits[4].header['NAXIS1'])
        # This is the normalised flux
        dr3_normalised_flux = ccd_fits[4].data
        # Uncertainties are saved on a relative scale in extension 0
        # Combining them with the normalised flux gives the standard deviation *sigma*
        dr3_normalised_error = ccd_fits[1].data * ccd_fits[4].data
        ccd_fits.close()
        
        # Interpolate from DR3 wavelength onto our common wavelength
        interpolated_normalised_flux = np.interp(
            x  = wavelengths_for_each_ccd['CCD'+ccd],
            xp = dr3_wavelength,
            fp = dr3_normalised_flux
        )
        normalised_flux_for_index.append(interpolated_normalised_flux)

        # Interpolate from DR3 wavelength onto our common wavelength
        interpolated_normalised_error = np.interp(
            x  = wavelengths_for_each_ccd['CCD'+ccd],
            xp = dr3_wavelength,
            fp = dr3_normalised_error
        )
        # instead of standard deviation, add 1/var == 1/std**2
        normalised_ivar_for_index.append(1./(interpolated_normalised_error)**2)
    
    normalised_flux_for_index = np.concatenate((normalised_flux_for_index))
    normalised_ivar_for_index = np.concatenate((normalised_ivar_for_index))
    
    return(normalised_flux_for_index,normalised_ivar_for_index)

In [ ]:
def populate_normalised_flux_and_ivar_matrix(matrix_index, wavelengths_for_each_ccd):
    sobject_id = training_set['sobject_id'][matrix_index]
    try:
        normalised_flux_for_index, normalised_ivar_for_index = load_normalised_spectra(sobject_id,wavelengths_for_each_ccd=wavelengths_for_each_ccd)
    except:
        print('Failed to load spectrum for index '+str(matrix_index)+', that is, sobject_id '+str(sobject_id))
    normalized_flux[matrix_index] = normalised_flux_for_index
    normalized_ivar[matrix_index] = normalised_ivar_for_index

# let's iterate over all matrix indices now
[populate_normalised_flux_and_ivar_matrix(matrix_index=index, wavelengths_for_each_ccd=wavelengths_for_each_ccd) for index in range(np.shape(training_set)[0])];

In [139]:
def create_mask_matrix(wavelength_array,labels):
    
    # First create a matrix, where all pixels are unmasked == 1 (used for fitting)
    mask_matrix = np.ones((np.shape(labels)[0],np.shape(wavelength_array)[0]))
    
    """
    To Do:
    - neglect cores of Halpha and Hbeta for stellar parameters
    - only use lines from Hinkle Atlas to avoid lines of element X being associated with element Y
    
    # Now for each label, figure out, which wavelength pixels we want to activate for fitting
    for label_index, label in enumerate(labels):
        if label in ['teff','logg','fe_h','vbroad']:
            mask_matrix[:,label_index] = 1
        if label in galah_elements:
            mask_matrix[:,label_index] = ... array with 1 for line region and 0 outside
    """
    return mask_matrix
mask_matrix = create_mask_matrix(wavelength_array,labels)

In [140]:
flux_ivar_file = open('training_sets/solar_twin_training_set_16xfe_snr50_flux_ivar.pickle','wb')
pickle.dump((normalized_flux,normalized_ivar),flux_ivar_file)
flux_ivar_file.close()